In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

## 准备数据

In [2]:
# 将数据转化为张量并进行标准化
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,),(0.5,))])

In [13]:
# 加载训练集和测试集
train_dataset = torchvision.datasets.MNIST(root='./data',train=True,download=True,transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data',train=False,download=True,transform=transform)

In [14]:
# 创建数据加载器(DataLoader),用于批量加载数据
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,batch_size = 64,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=64,shuffle=True)


## 定义模型


In [15]:
import torch.nn as nn
import torch.nn.functional as F

In [24]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel,self).__init__()
        # 灰度图输入通道为1
        # 输入数据格式为28x28的单通道灰度图
        self.conv1 = nn.Conv2d(1,32,kernel_size=5)
        self.conv2 = nn.Conv2d(32,64,kernel_size=5)
        self.fc1 = nn.Linear(1024,10)
        #输出层，10个分类
#         self.fc2 = nn.Linear(128,10)
        
    def forward(self,x):
        # 经过第一层卷积，池化后并激活
        # 池化后不改变通道数，第一层卷积输出通道数为32
        x = F.relu(F.max_pool2d(self.conv1(x),2))
        # 经过第二层卷积并激活
        x = F.relu(F.max_pool2d(self.conv2(x),2))
        # 展平
        x = x.view(-1,1024)
#         x = F.relu(self.fc1(x))
#         x = self.fc2(x)
        x = self.fc1(x)
        return x

输入为28x28的单通道灰度图,格式为(batch_size,1,28,28)

卷积层计算输出公式:
$ \\
    输出高度 = (输入高度-卷积核大小+2*{填充} / 步幅) +1
$
所以可得第一层卷积输出后为(batch_size,32,24,24)

池化层 :
$
    输出高度 = 输入高度 / 步幅
$
可得池化层输出大小为(batch_size,32,12,12)

计算可得第二个卷积生成的形状为batch_size,64,8,8 \\
经过池化后为batchs_size,64,4,4

* 展平的作用是因为全连接层只能接受一维向量 *

展平后有1024个特征

全连接层将展平后的特征输入进行线性变换，生成一个输出，
全连接层通过 z=W*x+b以及反向传播算法进行学习，具有全局视野。

## 定义损失函数和损失器

In [25]:
model = CNNModel()

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(),lr=0.01)

## 训练模型

In [26]:
num_epochs = 5

for epoch in range(num_epochs):
    running_loss = 0.0
    for i ,(images,labels) in enumerate(train_loader):
        #图像与标签传给模型
        outputs = model(images)
        loss = criterion(outputs,labels)
        
        
        #梯度清零
        optimizer.zero_grad()
        
        #反向传播梯度
        loss.backward()
        optimizer.step()
        running_loss=loss.item()
        
        if i %100 ==99:
            print(f'Epoch [{epoch+1}/{num_epochs}],step [{i+1}/{len(train_loader)}],loss: {running_loss/100:.4f}')
            running_loss=0.0
print('finished Training')

Epoch [1/5],step [100/938],loss: 0.0006
Epoch [1/5],step [200/938],loss: 0.0015
Epoch [1/5],step [300/938],loss: 0.0019
Epoch [1/5],step [400/938],loss: 0.0012
Epoch [1/5],step [500/938],loss: 0.0011
Epoch [1/5],step [600/938],loss: 0.0009
Epoch [1/5],step [700/938],loss: 0.0006
Epoch [1/5],step [800/938],loss: 0.0008
Epoch [1/5],step [900/938],loss: 0.0001
Epoch [2/5],step [100/938],loss: 0.0006
Epoch [2/5],step [200/938],loss: 0.0003
Epoch [2/5],step [300/938],loss: 0.0002
Epoch [2/5],step [400/938],loss: 0.0002
Epoch [2/5],step [500/938],loss: 0.0001
Epoch [2/5],step [600/938],loss: 0.0002
Epoch [2/5],step [700/938],loss: 0.0018
Epoch [2/5],step [800/938],loss: 0.0008
Epoch [2/5],step [900/938],loss: 0.0002
Epoch [3/5],step [100/938],loss: 0.0007
Epoch [3/5],step [200/938],loss: 0.0000
Epoch [3/5],step [300/938],loss: 0.0002
Epoch [3/5],step [400/938],loss: 0.0023
Epoch [3/5],step [500/938],loss: 0.0012
Epoch [3/5],step [600/938],loss: 0.0010
Epoch [3/5],step [700/938],loss: 0.0027


## 评估模型

In [27]:
correct =0
total=0

with torch.no_grad():
    for images,labels in test_loader:
        outputs = model(images)
        _,predicted = torch.max(outputs.data,1)
        total +=labels.size(0)
        correct +=(predicted==labels).sum().item()
print(f'Accuracy of the model on the test images:{100 * correct / total:.2f}%')

Accuracy of the model on the test images:97.55%
